# FW20 Trading System

## FW20 Futures

FW20 futures are standardized financial contracts traded on [the Warsaw Stock Exchange (GPW)](https://www.gpw.pl/en-home), where the underlying asset is the WIG20 index — a benchmark index representing the 20 largest companies on the Polish stock market.

### What Are FW20 Futures?

- FW20 stands for Futures on WIG20.
- These are cash-settled index futures, meaning there's no physical delivery of stocks—just a monetary settlement based on the index value.
- They allow investors to speculate, hedge, or gain leveraged exposure to the Polish equity market.

### Contract Specifications

- **Underlying asset**: WIG20 index
- **Multiplier**: 20 PLN per index point → If the contract is priced at 2,000 points, its nominal value is 40,000 PLN.
- **Settlement**: Cash only, no delivery of shares
- **Expiration months**: March, June, September, December (quarterly cycle)
- **Last trading day**: Third Friday of the expiration month (or the prior session if Friday is a holiday)

### Contract Symbol: FW20EYY20

- **F**: Futures contract
- **W20**: Underlying assest - WIG20 index
- **E**: Expiration month - H (March), M (June), U (September), Z (December)
- **YY**: Year of expiration
- **20**: Multiplier
- **Example**: FW20H2620 indicates futures contract expiring in June 2026

### How They’re Used

- **Speculation**: Traders bet on the direction of the WIG20 index.
- **Hedging**: Investors protect their portfolios against market downturns.
- **Arbitrage**: Exploit price differences between futures and spot markets.
- **Leverage**: Control a large position with relatively small capital.

### Risks

- Futures are highly leveraged, so small index movements can lead to large gains or losses.
- They require margin deposits and are marked-to-market daily.
- Not suitable for beginners without proper risk management.

### More Info

- [GPW's derivative page](https://www.gpw.pl/derivatives)
- [BOSSA's futures education section](https://bossa.pl/edukacja/kontrakty-opcje/kontrakty-terminowe/charakterystyka/kontrakty-na-indeksy) [PL version only]



## EDA

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [56]:
filepath = "data/fw20.csv"
df = pd.read_csv(filepath, parse_dates=['Date'])
df.set_index('Date', inplace=True) # Set 'Date' as the index
df.sort_index(inplace=True) # Ensure data is sorted by date

In [57]:
df.tail()

,Open,High,Low,Close,Volume,Open Interest
Date,,,,,,
2025-09-25,2853,2856,2815,2825,20231,62808.0
2025-09-26,2828,2863,2813,2863,17378,65270.0
2025-09-29,2873,2880,2845,2859,13546,65238.0
2025-09-30,2854,2857,2830,2857,14266,66112.0
2025-10-01,2844,2885,2831,2878,18432,67739.0


In [58]:
df.isnull().sum()

Open             0
High             0
Low              0
Close            0
Volume           0
Open Interest    2
dtype: int64

In [59]:
df['Open Interest'].ffill(inplace=True)
df['Open Interest'] = df['Open Interest'].astype(int)

C:\Users\micha\AppData\Local\Temp\ipykernel_21488\1358777520.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Open Interest'].ffill(inplace=True)


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6932 entries, 1998-01-20 to 2025-10-01
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Open           6932 non-null   int64
 1   High           6932 non-null   int64
 2   Low            6932 non-null   int64
 3   Close          6932 non-null   int64
 4   Volume         6932 non-null   int64
 5   Open Interest  6932 non-null   int64
dtypes: int64(6)
memory usage: 379.1 KB


In [61]:
# calculate ATR
short_window = 2
long_window = 14

df['Prev_Close'] = df['Close'].shift(1).bfill().astype(int)
df['H-L'] = (df['High'] - df['Low']).astype(int)
df['H-PC'] = abs(df['High'] - df['Prev_Close']).astype(int)
df['L-PC'] = abs(df['Low'] - df['Prev_Close']).astype(int)
df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
df['ATR_short'] = df['TR'].rolling(window=short_window).mean()
df['ATR_long'] = df['TR'].rolling(window=long_window).mean()

In [62]:
df.head()


,Open,High,Low,Close,Volume,Open Interest,Prev_Close,H-L,H-PC,L-PC,TR,ATR_short,ATR_long
Date,,,,,,,,,,,,,
1998-01-20,1440,1440,1440,1440,4,12,1440,0,0,0,0,NaN,NaN
1998-01-22,1370,1370,1370,1370,9,21,1440,0,70,70,70,35.0,NaN
1998-01-23,1370,1370,1370,1370,2,22,1370,0,0,0,0,35.0,NaN
1998-01-28,1382,1390,1382,1389,8,9,1370,8,20,12,20,10.0,NaN
1998-01-29,1417,1417,1412,1412,4,21,1389,5,28,23,28,24.0,NaN


In [63]:
df.tail()

,Open,High,Low,Close,Volume,Open Interest,Prev_Close,H-L,H-PC,L-PC,TR,ATR_short,ATR_long
Date,,,,,,,,,,,,,
2025-09-25,2853,2856,2815,2825,20231,62808,2861,41,5,46,46,53.0,45.285714
2025-09-26,2828,2863,2813,2863,17378,65270,2825,50,38,12,50,48.0,44.500000
2025-09-29,2873,2880,2845,2859,13546,65238,2863,35,17,18,35,42.5,43.714286
2025-09-30,2854,2857,2830,2857,14266,66112,2859,27,2,29,29,32.0,40.357143
2025-10-01,2844,2885,2831,2878,18432,67739,2857,54,28,26,54,41.5,40.071429


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6932 entries, 1998-01-20 to 2025-10-01
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Open           6932 non-null   int64  
 1   High           6932 non-null   int64  
 2   Low            6932 non-null   int64  
 3   Close          6932 non-null   int64  
 4   Volume         6932 non-null   int64  
 5   Open Interest  6932 non-null   int64  
 6   Prev_Close     6932 non-null   int64  
 7   H-L            6932 non-null   int64  
 8   H-PC           6932 non-null   int64  
 9   L-PC           6932 non-null   int64  
 10  TR             6932 non-null   int64  
 11  ATR_short      6931 non-null   float64
 12  ATR_long       6919 non-null   float64
dtypes: float64(2), int64(11)
memory usage: 758.2 KB


In [65]:
# generate signals
signal_adj = 0.6

df['Signal'] = (df['ATR_short'] * signal_adj).bfill().astype(int)
df['Buy_Level'] = (df['Open'] + df['Signal']).astype(int)
df['Sell_Level'] = (df['Open'] - df['Signal']).astype(int)

# Define entry conditions
df['Long_Entry'] = df['High'] > df['Buy_Level']
df['Short_Entry'] = df['Low'] < df['Sell_Level']

# --- Simulate Trades ---
# Initialize position: 1 = long, -1 = short, 0 = flat
df['Position'] = 0
df.loc[df['Long_Entry'], 'Position'] = 1
df.loc[df['Short_Entry'], 'Position'] = -1

In [66]:
df.head()

,Open,High,Low,Close,Volume,Open Interest,Prev_Close,H-L,H-PC,L-PC,TR,ATR_short,ATR_long,Signal,Buy_Level,Sell_Level,Long_Entry,Short_Entry,Position
Date,,,,,,,,,,,,,,,,,,,
1998-01-20,1440,1440,1440,1440,4,12,1440,0,0,0,0,NaN,NaN,21,1461,1419,False,False,0
1998-01-22,1370,1370,1370,1370,9,21,1440,0,70,70,70,35.0,NaN,21,1391,1349,False,False,0
1998-01-23,1370,1370,1370,1370,2,22,1370,0,0,0,0,35.0,NaN,21,1391,1349,False,False,0
1998-01-28,1382,1390,1382,1389,8,9,1370,8,20,12,20,10.0,NaN,6,1388,1376,True,False,1
1998-01-29,1417,1417,1412,1412,4,21,1389,5,28,23,28,24.0,NaN,14,1431,1403,False,False,0
